<a href="https://colab.research.google.com/github/bokutachi256/gisday2016/blob/master/estat_api_test_python36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArcPyを用いたWeb APIの活用（Python3.6版） 
　
- GIS Day in 東京 2016 Eコース
- 首都大学東京 都市環境学部 地理環境コース 中山大地
- 2016年10月29日 首都大学東京 南大沢キャンパス

# はじめに

この講習会でfはe-Stat API バージョン 2.1に準拠しています．
e-Stat APIに関しての詳しい情報は，
[e-Stat API仕様のページ (http://www.e-stat.go.jp/api/api-spec/)](http://www.e-stat.go.jp/api/api-spec/)をご覧下さい．

# ねらい

この講習会では，ArcPyからe-statを効率的に利用する方法について学びます．
具体的には国勢調査の小地域テーブルをe-Stat APIを用いて取得し，
ArcGISに取り込むことを目的としています．

下の例は，e-Statの[地図で見る統計（統計GIS）(https://www.e-stat.go.jp/SG1/estat/redirect.do?url=toukeiChiri.do%3Fmethod%3Dinit)](https://www.e-stat.go.jp/SG1/estat/redirect.do?url=toukeiChiri.do%3Fmethod%3Dinit)
から取得した東京都の平成22年国勢調査（小地域）年齢別（5歳階級，4区分），
男女別人口のデータになります（**tblT000573C13101.xlsx**参照）．

列名と意味は以下のようになっています．
- `KEY_CODE`：地域コード
- `HYOSYO`：地域の住所レベル
- `CITYNAME`：市区町村名
- `NAME`：小地域名（町丁目名）
- `HTKSYORI`：秘匿処理がされているか
- `HTKSAKI`：秘匿処理の処理先
- `GASSAN`：合算処理がなされているか
- `T000573001`から`T000573060`：年齢別・男女別カテゴリの人口

KEY_CODE|HYOSYO|CITYNAME|NAME|HTKSYORI|HTKSAKI|GASSAN|T000573001|T000573002|省略|T000573060
-------:|-----:|:-------|:---|:-------|------:|-----:|---------:|---------:|:--:|---------:
|||||||総数、年齢「不詳」含む|総数０〜４歳|・・・|女７５歳以上
13101|1|千代田区|||||47115|1688|・・・|2882
131010010|2|千代田区|丸の内||||X|X|・・・|X
13101001001|3|千代田区|丸の内１丁目|秘匿地域|140||X|X|・・・|X
省略|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・
13101003001|3|千代田区|内幸町１丁目|秘匿地域|4001||X|X|・・・|X
13101003002|3|千代田区|内幸町２丁目||||-|-|・・・|-
131010040|2|千代田区|有楽町||||33|-|・・・|4
13101004001|3|千代田区|有楽町１丁目|合算地域あり||003001;004002|33|-|・・・|4
13101004002|3|千代田区|有楽町２丁目|秘匿地域|4001||X|X|・・・|X
省略|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・
131010180|2|千代田区|富士見||||2758|110|・・・|123
13101018001|3|千代田区|富士見１丁目||||770|19|・・・|47
13101018002|3|千代田区|富士見２丁目||||1988|91|・・・|76
省略|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・|・・・

通常の利用ですと，このデータを取得するには統計GISからメニューをたどり，
市区町村別にデータをダウンロードしなくてはなりません．
必要なデータが少量ならこの方法でもよいのですが，
広い範囲のデータが必要な時には繁雑な作業になります．

e-Stat APIを用いることにより，
プログラムからデータを取得することが可能になります．
広い範囲のデータを一度に取得したり，
ローカルにデータを持たずにデータ解析を行うことができます．

# 作業を始めるにあたって

# e-Stat APIキーの取得

e-Stat APIを使用するためには，
[e-Stat APIのページ (http://www.e-stat.go.jp/api/)](http://www.e-stat.go.jp/api/)からユーザー登録を行い，
APIキーを取得する必要があります．
ここでは，講習会用に取得したキーを共用します．

共用するキーは以下になります．
```
c8fbd72d8f58aedf4ca3106034601d0ea83e9a69
```

# e-Stat APIを利用する手順

e-Stat APIを用いて統計表を取得するには，以下の手順が必要になります．

1. 統計表情報取得用APIを用いて条件に合う統計表の有無を調べ，統計表IDを取得する．
2. メタ情報取得用APIを用いて統計表の表章事項（項目名など）や地域情報を取得し，
表の外枠を作成する．
3. 統計データ取得APIを用いて統計表の本体データを取得する．
4. データを整形して出力する．

# Pythonモジュールのインポート

Pythonのモジュールをインポートします．
各モジュールは以下の役割があります．

- `json`：jsonを扱うためのモジュール
- `urllib`：urlによる各種リソースへのアクセスをするモジュール
- `csv`：CSVファイル操作を扱うモジュール
- `sys`：Pythonのシステムパラメータに関するモジュール
- `os`：OSの機能にアクセスするためのモジュール
- `codecs`：ユニコードエスケープに必要なモジュール

まずは必要なモジュールをインポートしましょう．

In [0]:
# -`- coding: utf-8 -`-

import json
import urllib.parse
import urllib.request
import csv
import sys
import os
import codecs

from tqdm import tqdm
import time

# e-Stat APIについて

e-Stat APIには7個のAPIがあります．
ここでは以下の3個を使います．

- 統計表情報取得用API："http://api.e-stat.go.jp/rest/2.1/app/json/getStatsList"
- メタ情報取得用API："http://api.e-stat.go.jp/rest/2.1/app/json/getMetaInfo"
- 統計データ取得用API："http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData"

各APIはAPIのURLを用いてサーバにアクセスし，GETメソッドを使ってデータの取得を行います．
GETメソッドでは，APIのURL末尾にに"?"をつけ，
`キー = 値`の形でパラメータを引き渡します．
パラメータが複数ある場合には，`キー = 値`のペアを"&"で区切って指定します．

# 統計表情報の取得

まずは統計表情報取得用APIを用いて
条件に合う統計表が提供されているか調べてみましょう．
APIのURLは長いので，変数にURLを格納して使用します．

- `APIlist_API`：統計表情報取得用API
- `APImeta_URL`：メタ情報取得用API
- `APIdata_URL`：統計データ取得用API

各URLはパラメータ引き渡し用の"?"を末尾に追加してあります．

In [0]:
APIlist_URL='http://api.e-stat.go.jp/rest/2.1/app/json/getStatsList?'
APImeta_URL='http://api.e-stat.go.jp/rest/2.1/app/json/getMetaInfo?'
APIdata_URL='http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'

# APIパラメータの設定

次にAPIに送るパラメータ文字列を作成しましょう．
パラメータはAPIのURLに続いて以下の形式で指定します．

```
キー = 値
```

複数のキーと値を渡す場合にはキーと値のペアを`&`で区切ります．
またパラメータはURLエンコードする必要があります．
ここではキーと値のペアをディクショナリ型で指定し，
`urllib.urlencode`関数を用いてURLエンコードします．

ディクショナリ型の指定はキーと値を":"で区切って羅列します．

```
ディクショナリ名 = {
    キー1 : 値1，
    キー2 : 値2
}
```
これを`urllib.urlencode`関数に渡すと，
自動的にGETに適した形の文字列に変換してくれます．

APIマニュアル**「3.2. 統計表情報取得」**より，
統計表情報取得用APIではアプリケーションIDを表すパラメータ`appId`のみが必須で，
これ以外のパラメータは任意となっています．
まずは2010年調査のデータのうち
表題やメタ情報などに"国勢調査"，"年齢別"，"東京"を含む
小地域データを検索してみましょう．
調査年を表すパラメータは`surveyYears`，
検索文字列を表すパラメータは`searchWord`，
データ種別を表すパラメータは`searchKind`
になります．
これらをディクショナリ型のオブジェクト`param`に格納します．

ディクショナリ型のオブジェクトではAPIのパラメータに相当するものは"キー"となります．
キー名に文字列を使う場合はクオーテーションで囲む必要があり，
値が文字列の場合はこちらもクオーテーションで囲みます．
パラメータの順番を気にする必要はありません．

まずはAPIにアクセスするためのアプリケーションIDをセットしましょう．
次に調査年，キーワードと続きます．
キーワードは複数あり，すべてのキーワードを満たすものを取得したいため`AND`で区切ります．
データ種別は小地域になるため`2`になります．

In [0]:
param = {
    'appId': 'c8fbd72d8f58aedf4ca3106034601d0ea83e9a69',
    'surveyYears': 2010,
    'searchWord': '国勢調査 AND 年齢別 AND 東京',
    'searchKind': 2
}

オブジェクト`param`にパラメータがセットできたら
`urllib.urlencode`を使ってURLエンコードし，
`api_param`に代入しておきます．

In [0]:
api_param = urllib.parse.urlencode(param)

念のため`api_param`を表示してきちんと設定できているか確認しましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
api_param

'appId=c8fbd72d8f58aedf4ca3106034601d0ea83e9a69&surveyYears=2010&searchWord=%E5%9B%BD%E5%8B%A2%E8%AA%BF%E6%9F%BB+AND+%E5%B9%B4%E9%BD%A2%E5%88%A5+AND+%E6%9D%B1%E4%BA%AC&searchKind=2'

各パラメータは`&`で区切られ，
値の文字列がURLエンコードされていることがわかります．
パラメータの順序が変わる場合がありますが，
気にする必要はありません．

# APIへのアクセス

それでは実際に統計表情報取得用APIにアクセスしましょう．
APIへのアクセスURLは`APIlist_URL`と`api_param`を結合したものになります．
Pythonでは文字列（文字列変数も含む）の結合に"+"を使用します．
すなわち，`APIlist_URL + api_param`でパラメータを含むAPIへのURLができます．
試しに`APIlist_URL + api_param`を確認してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
APIlist_URL + api_param

'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsList?appId=c8fbd72d8f58aedf4ca3106034601d0ea83e9a69&surveyYears=2010&searchWord=%E5%9B%BD%E5%8B%A2%E8%AA%BF%E6%9F%BB+AND+%E5%B9%B4%E9%BD%A2%E5%88%A5+AND+%E6%9D%B1%E4%BA%AC&searchKind=2'

統計表情報取得用APIのURLとエンコードされたパラメータが結合されたことがわかります．

パラメータを含むAPIへのURLを`urllib.urlopen`関数に渡すと
サーバへのコネクションが発生し，戻り値としてパラメーターで
指定した情報へのオブジェクトが返されます．

返ってきたオブジェクトは`data_obj`に格納します．

In [0]:
data_obj = urllib.request.urlopen(APIlist_URL + api_param)

# 検索結果の取得

返ってきたオブジェクトに対して`read`メソッドを使い，
検索結果を取得します．
検索結果はオブジェクト`response`に格納します．

In [0]:
response = data_obj.read()

`response`に格納したjsonデータを`json.loads`関数を用いてPythonのデータ型に変換します．

In [0]:
json_data = json.loads(response)

これでオブジェクト`json_data`に統計表情報取得用APIの結果が格納されました．
うまく取得できているか`json_data`の内容を確認してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
json_data

{'GET_STATS_LIST': {'RESULT': {'STATUS': 0,
   'ERROR_MSG': '正常に終了しました。',
   'DATE': '2018-10-06T12:51:42.750+09:00'},
  'PARAMETER': {'LANG': 'J',
   'SURVEY_YEARS': 2010,
   'SEARCH_WORD': '国勢調査 AND 年齢別 AND 東京',
   'SEARCH_KIND': 2,
   'DATA_FORMAT': 'J'},
  'DATALIST_INF': {'NUMBER': 1,
   'RESULT_INF': {'FROM_NUMBER': 1, 'TO_NUMBER': 1},
   'TABLE_INF': {'@id': 'T00057313',
    'STAT_NAME': {'@code': '00200521', '$': '国勢調査'},
    'GOV_ORG': {'@code': '00200', '$': '総務省'},
    'STATISTICS_NAME': '平成２２年国勢調査(小地域)20101001',
    'TITLE': {'@no': '13', '$': '年齢別（５歳階級、４区分）、男女別人口 東京都'},
    'CYCLE': '-',
    'SURVEY_DATE': 201010,
    'OPEN_DATE': '2013-06-10',
    'SMALL_AREA': 1,
    'MAIN_CATEGORY': {'@code': '02', '$': '人口・世帯'},
    'SUB_CATEGORY': {'@code': '01', '$': '人口'},
    'OVERALL_TOTAL_NUMBER': 406860,
    'UPDATED_DATE': '2013-06-10',
    'STATISTICS_NAME_SPEC': {'TABULATION_CATEGORY': '平成２２年国勢調査(小地域)20101001'},
    'TITLE_SPEC': {'TABLE_NAME': '年齢別（５歳階級、４区分）、男女別人口',
     'TA

意味のわからない結果が表示されてしまいました．

APIマニュアルにも記述がありますが，
e-Stat APIをjson型式で使用した場合，
文字がユニコードエスケープされた状態で返されます．
このため文字化けしたような状態で表示されます．
また，適切なインデントがされていないため可読性が悪くなっています．
これでは結果のチェックができないため，
`json.dumps`を使ってユニコードエスケープの解除とインデントの設定を行います．

`json.dumps`の書式は以下になります．
```
json.dumps(変換したいオブジェクト, ensure_ascii = False, indent = インデント数)
```

`ensure_ascii`は，非ASCII文字をユニコードエスケープするかどうかの指定です．
デフォルトでは`True`になっているためにユニコードエスケープされますので，
`False`にしてエスケープしない設定にします．
インデント数は好きな値にしてください．

ユニコードエスケープの解除とインデントを設定して表示してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(json.dumps(json_data, ensure_ascii = False, indent = 4))

{
    "GET_STATS_LIST": {
        "RESULT": {
            "STATUS": 0,
            "ERROR_MSG": "正常に終了しました。",
            "DATE": "2018-10-06T12:51:42.750+09:00"
        },
        "PARAMETER": {
            "LANG": "J",
            "SURVEY_YEARS": 2010,
            "SEARCH_WORD": "国勢調査 AND 年齢別 AND 東京",
            "SEARCH_KIND": 2,
            "DATA_FORMAT": "J"
        },
        "DATALIST_INF": {
            "NUMBER": 1,
            "RESULT_INF": {
                "FROM_NUMBER": 1,
                "TO_NUMBER": 1
            },
            "TABLE_INF": {
                "@id": "T00057313",
                "STAT_NAME": {
                    "@code": "00200521",
                    "$": "国勢調査"
                },
                "GOV_ORG": {
                    "@code": "00200",
                    "$": "総務省"
                },
                "STATISTICS_NAME": "平成２２年国勢調査(小地域)20101001",
                "TITLE": {
                    "@no": "13",
                    "$": "年齢別（５歳階級、４区分）、

うまくいきました．

以下のコードを実行すると，
統計表情報取得用APIの結果を`APIlist.json`として保存することができます．
保存した結果を[JSON EDITOR ONLINE (http://jsoneditoronline.org/)](http://jsoneditoronline.org/)
などで表示してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
f = codecs.open('APIlist.json', 'w', 'utf-8')
f.write(json.dumps(json_data, ensure_ascii = False, indent = 4))
f.close()

>`codecs.open`関数は文字コードの指定付きでファイルをオープンします．
ここでは`APIlist_json.txt`というファイルを書き込みモード`w`で開き，
文字コードをUTF-8に設定します．
開いたファイルへのポインタはオブジェクト`f`に格納します．
実際にファイルに書き込むためにはオブジェクト`f`に対して`write`メソッドを使います．
書き込み終了後は`close`メソッドを使ってファイルを閉じます．

# API出力データの検討

それではAPIマニュアル**「4. APIの出力データ」**を参考にしてAPIの出力データを検討してみましょう．

ルートタグは`GET_STATS_LIST`で，
その下に子タグとして`DATALIST_INF`，`PARAMETER`，`RESULT`タグがあります．
処理によっては子タグの順番が変わることもあります．

まずは`RESULT`タグを見てみます．

```
"RESULT": {
    "STATUS": 0, 
    "DATE": "2016-10-20T13:47:49.159+09:00", 
    "ERROR_MSG": "正常に終了しました。"
}
```

APIマニュアル**「4.1.1 RESULTタグ」**より，
`RESULT`タグにはAPIの出力結果情報が出力されています．
`RESULT`タグの中の`DATE`タグは結果が出力された日時，
`STATUS`は処理結果のコードで，`ERROR_MSG`が結果に対応するメッセージです．
APIマニュアルより，`STATUS`は`0`になっており，APIが正常に終了していることがわかります．

次に`PARAMETER`タグです．
APIマニュアル**「4.2.1 PARAMETERタグ」**より，
このタグはAPIを呼び出したときのパラメータが格納されています．

```
"PARAMETER": {
    "LANG": "J", 
    "SURVEY_YEARS": 2010, 
    "SEARCH_WORD": "国勢調査 AND 年齢別 AND 東京", 
    "DATA_FORMAT": "J", 
    "SEARCH_KIND": 2
},
```

結果の本体は`DATALIST_INF`タグです．
APIマニュアル**「4.2.2 DATALIST_INFタグ」**より，
ここで重要なのは`TABLE_INF`タグで，
統計表のさまざまな属性が入っています．

```
"TABLE_INF": {
    "SUB_CATEGORY": {
        "@code": "01", 
        "$": "人口"
    }, 
    "SMALL_AREA": 1, 
    "STATISTICS_NAME": "平成２２年国勢調査(小地域)20101001", 
    "GOV_ORG": {
        "@code": "00200", 
        "$": "総務省"
    }, 
    "STAT_NAME": {
        "@code": "00200521", 
        "$": "国勢調査"
    }, 
    "TITLE": {
        "$": "年齢別（５歳階級、４区分）、男女別人口 東京都", 
        "@no": "13"
    }, 
    "MAIN_CATEGORY": {
        "@code": "02", 
        "$": "人口・世帯"
    }, 
    "SURVEY_DATE": 201010, 
    "STATISTICS_NAME_SPEC": {
        "TABULATION_CATEGORY": "平成２２年国勢調査(小地域)20101001"
    }, 
    "UPDATED_DATE": "2013-06-10", 
    "OPEN_DATE": "2013-06-10", 
    "OVERALL_TOTAL_NUMBER": 406860, 
    "@id": "T00057313", 
    "TITLE_SPEC": {
    "TABLE_SUB_CATEGORY1": "東京都", 
    "TABLE_NAME": "年齢別（５歳階級、４区分）、男女別人口"
    }, 
    "CYCLE": "-"
}
```

特に統計表のID（統計表ID）の`@id`，
統計表名の提供統計名`STATISTICS_NAME`，
統計表の表題`TITLE`はチェックする必要があります．
また，統計データの総件名`OVERALL_TOTAL_NUMBER`も重要です．

これらの情報のみを表示してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['@id'])
print(json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['STATISTICS_NAME'])
print(json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['OVERALL_TOTAL_NUMBER'])
print(json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['TITLE_SPEC']['TABLE_NAME'])
print(json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['TITLE_SPEC']['TABLE_SUB_CATEGORY1'])

T00057313
平成２２年国勢調査(小地域)20101001
406860
年齢別（５歳階級、４区分）、男女別人口
東京都


これで必要な情報が確認できました．

JSONの表記，Pythonでの表記，Pythonでの要素へのアクセス方法は以下のようになっています．
なお，Pythonではダブルクオーテーションとシングルクオーテーションは同じ役割をします．

Pythonでの<br>オブジェクト型|JSON型式での表記|Pythonでの表記例|Pythonでの<br>アクセス方法|結果
-|-
ディクショナリ型|`{`<br>&emsp;`"要素名1": "文字型値",`<br>&emsp;`"要素名2": 数値`<br>`}`|`json_data = {`<br>&emsp;`"NAME": "東京",`<br>&emsp;`"CODE": 13`<br>`}`|`json_data["NAME"]`<br>`json_data["CODE"]`|&rarr;東京<br>&rarr;13
親子要素のある<br>ディクショナリ型|`{`<br>&emsp;`"親要素名":{`<br>&emsp;&emsp;`"子要素名1": "文字型値",`<br>&emsp;&emsp;`"子要素名2": 数値`<br>&emsp;`}`<br>`}`|`json_data = {`<br>&emsp;`"TABLE_INF":{`<br>&emsp;&emsp;`"NAME": "東京",`<br>&emsp;&emsp;`"CODE": 13`<br>&emsp;`}`<br>`}`|`json_data["TABLE_INF"]["NAME"]`<br>`json_data["TABLE_INF"]["CODE"]`|&rarr;東京<br>&rarr;13
入れ子のディクショナリ型|`{`<br>&emsp;`"親要素名":{`<br>&emsp;&emsp;`"子要素名1":{`<br>&emsp;&emsp;&emsp;`"子要素名2": "文字型値",`<br>&emsp;&emsp;&emsp;`"子要素名3": 数値`<br>&emsp;&emsp;`}`<br>&emsp;`}`<br>`}`|`json_data = {`<br>&emsp;`"META_INF":{`<br>&emsp;&emsp;`"LOC_INFO":{`<br>&emsp;&emsp;&emsp;`"NAME": "東京",`<br>&emsp;&emsp;&emsp;`"CODE": 13`<br>&emsp;&emsp;`}`<br>&emsp;`}`<br>`}`|`json_data["META_INF"]["LOC_INFO"]["NAME"]`<br>`json_data["META_INF"]["LOC_INFO"]["CODE"]`|&rarr;東京<br>&rarr;13
リスト型|`["値1", "値2", "値2", "値4"]`|`json_data = [`<br>&emsp;`"埼玉",`<br>&emsp;` "千葉",`<br>&emsp;` "東京",`<br>&emsp;` "神奈川"`<br>`]`|`json_data[0]`<br>`json_data[3]`|&rarr;埼玉<br>&rarr;神奈川

>ディクショナリ型オブジェクトとリスト型オブジェクトはともに配列変数の一種です．ディクショナリ型オブジェクトは配列の添え字に文字列などが使えますが，リスト型オブジェクトは数値（基本的に0以上の整数値）のみしか使えません．ディクショナリ型オブジェクトの添え字は「キー」と呼ばれ，キーに対応する値とのペアをコロン「`:`」でつなげたものをカンマ「`,`」で区切り，全体をブレース「`{}`」で囲みます．他のプログラミング言語では，ディクショナリ型オブジェクトに相当する配列を連想配列，ハッシュ，マップなどと呼ぶこともあります．リスト型オブジェクトの添え字は数値のみで，値のみをカンマ「`,`」で区切り，全体をスクエアブラケット「`[]`」で囲みます．リストの添え字は明示的には指定せず，羅列した最初の値の添え字がが`0`になり，1ずつ添え字が増えていきます．

# 必要な情報の切り出し

APIから取得したデータには不要な部分も多く含まれているため，
必要なデータのみを新しいオブジェクトに入れておきましょう．

ここではディクショナリ型のオブジェクト`tab_info`に上記の情報のみを入れておきます．

In [0]:
tab_info = {
    '@id': json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['@id'],
    'STATISTICS_NAME': json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['STATISTICS_NAME'],
    'OVERALL_TOTAL_NUMBER': json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['OVERALL_TOTAL_NUMBER'],
    'TABLE_NAME': json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['TITLE_SPEC']['TABLE_NAME'],
    'TABLE_SUB_CATEGORY1': json_data['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']['TITLE_SPEC']['TABLE_SUB_CATEGORY1']
}

`tab_info`を確認します．

オブジェクト`tab_info`はjsonからコピーしたものになりますので，
文字列はユニコードエスケープされたままです．
前述の`json.dumps`を用いて整形して表示します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(json.dumps(tab_info, ensure_ascii = False, indent = 4))

{
    "@id": "T00057313",
    "STATISTICS_NAME": "平成２２年国勢調査(小地域)20101001",
    "OVERALL_TOTAL_NUMBER": 406860,
    "TABLE_NAME": "年齢別（５歳階級、４区分）、男女別人口",
    "TABLE_SUB_CATEGORY1": "東京都"
}


うまくいきました．

# 統計表メタデータの取得

欲しい統計表が見つかったら，次は統計表のメタデータを取得しましょう．
メタデータには統計表の表章事項，分類事項，地域情報などが含まれます．

メタデータの取得にはメタ情報取得用APIを使用します．
APIマニュアル**「3.3. メタ情報取得」**より，
このAPIの必須パラメータはアプリケーションIDの`appId`，
統計表IDの`statsDataId`です．
ディクショナリ型でこれらのパラメータを表しましょう．

In [0]:
param = {
    'appId': 'c8fbd72d8f58aedf4ca3106034601d0ea83e9a69',
    'statsDataId': tab_info['@id']
}

統計表IDはオブジェクト`tab_info`のキー`@id`に入っているため，
```
tab_info['@id']
```
で統計表IDを取得できます．

これらのパラメータを`urllib.urlencode`でURLエンコードし，メタデータ取得用APIに渡します．

In [0]:
api_param = urllib.parse.urlencode(param)
data_obj = urllib.request.urlopen(APImeta_URL + api_param)
response = data_obj.read()
json_data = json.loads(response)

`json.dumps`を用いて結果を確認してみましょう．
かなりの量の結果になっています．
以下では途中を省略して表示します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#print(json.dumps(json_data, ensure_ascii = False, indent = 4))

```
{
    "GET_META_INFO": {
        "METADATA_INF": {
            "CLASS_INF": {
                "CLASS_OBJ": [
                    {
                        "@id": "cat01", 
                        "@name": "年齢別（５歳階級、４区分）、男女別人口", 
                        "CLASS": [
・・・途中省略・・・
                        ]
                    }, 
                    {
                        "@id": "cat02", 
                        "@name": "秘匿地域・合算地域有り", 
                        "CLASS": [
・・・途中省略・・・
                        ]
                    }, 
                    {
                        "@id": "area", 
                        "@name": "東京都", 
                        "CLASS": [
・・・途中省略・・・
                        ]
                    }
                ]
            }, 
            "TABLE_INF": {
・・・途中省略・・・
            }
        }, 
        "PARAMETER": {
・・・途中省略・・・
        }, 
        "RESULT": {
・・・途中省略・・・
        }
    }
}
```

以下のコードを実行すると，
メタ情報取得用APIの結果を`APImeta.json`として保存することができます．
保存した結果を[JSON EDITOR ONLINE (http://jsoneditoronline.org/)](http://jsoneditoronline.org/)
などで表示してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
f = codecs.open('APImeta.json', 'w', 'utf-8')
f.write(json.dumps(json_data, ensure_ascii = False, indent = 4))
f.close()

# メタデータの確認

取得したデータを確認してみます．
APIマニュアルの「**4.3 メタ情報取得**」も合わせて参考にします．
戻ってきたデータはかなり複雑ですが，
丁寧に見ていきながら解釈をしていきましょう．

取得したメタ情報は以下のようになっています．
```
{
    "GET_META_INFO": {
        "METADATA_INF": {
・・・途中省略・・・
        "PARAMETER": {
・・・途中省略・・・
        "RESULT": {
・・・途中省略・・・
```

ルートタグは`GET_META_INFO`で，
その下に子タグとして`METADATA_INF`と`PATAMETER`と`RESULT`があります．
`RESULT`はAPIリクエストの結果，`PARAMETER`はリクエスト時に
指定したパラメータを出力したものです．
肝心の情報は`METADATA_INF`に入っています．

`METADATA_INF`は以下のようになっています．

```
"METADATA_INF": {
    "CLASS_INF": {
        "CLASS_OBJ": [
・・・途中省略・・・
    }
    "TABLE_INF": {
        "SUB_CATEGORY": {
・・・以下省略・・・
```

`METADATA_INF`には子タグとして`CLASS_INF`と`TABLE_INF`があり，
`TABLE_INF`は統計表の情報が，`CLASS_INF`には統計データのメタデータが入っています．
ここで重要なのは`CLASS_INF`タグの中の`CLASS_OBJ`タグです．
これを詳細に見ていきましょう．

APIマニュアルの**｢4.3.2. METADATA_INFタグ」**によると，
`CLASS_OBJ`タグはメタ情報の数だけ出力するとあります．
それでは`CLASS_OBJ`タグには何個のメタ情報があるのでしょうか．


`CLASS_OBJ`タグは直後にスクエアブラケット`[]`が続いているため，
ディクショナリ型ではなくリスト型のオブジェクトです．
リスト型オブジェクトに含まれている要素の数を知るためには
`len`関数を使います．

`CLASS_OBJ`へのアクセスは
`json_data["GET_META_INFO"]["METADATA_INF"]["CLASS_INF"]["CLASS_OBJ"]`
になるため，
`CLASS_OBJ`の要素数は次のようにして取得できます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(len(json_data['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']))

3


`CLASS_OBJ`の要素数は3個であることがわかりました．

`CLASS_OBJ`までのアクセスが長いため，
ちょっと扱いが厄介です．
`CLASS_OBJ`以下の部分をオブジェクト`class_obj`に切り出します．

In [0]:
class_obj = json_data['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

念のため，いま切り出した`class_obj`の要素数を確認します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
len(class_obj)

3

先ほどと同様に3個になっていることが確認できました．
ちゃんと切り出しができているようです．

# CLASS_OBJの確認

次にオブジェクト`class_obj`に格納した`CLASS_OBJ`の内容を確認していきます．
`CLASS_OBJ`はリスト型オブジェクトなので，
各要素へのアクセスは`0`から始まる要素番号で指定します．
`CLASS_OBJ`の要素数は3個なので，要素番号は`0`，`1`，`2`の3個になります．
すなわち，
```
class_obj[0]
class_obj[1]
class_obj[2]
```
になります．
簡単に書くと以下のようになります．

```
"CLASS_OBJ": [
    1個目の要素（class_obj[0]の値）,
    2個目の要素（class_obj[1]の値）,
    3個目の要素（class_obj[2]の値）
]
```

実際にはCLASS_OBJの各要素は{}で囲まれていることから，
ディクショナリ型のオブジェクトが入れ子になっていることがわかります．

```
"CLASS_OBJ": [
    {
        1個目の要素（値としてディクショナリ型）
    },
    {
        2個目の要素（値としてディクショナリ型）
    },
    {
        3個目の要素（値としてディクショナリ型）
    }
]
```

それぞれのディクショナリ型の要素の中には何個の要素があるのでしょうか．
ディクショナリ型の要素数を知るには，リスト型と同様に`len`関数を使います．
リストの1個目にアクセスするためには要素番号を`0`にしますので，
要素0から要素2までの要素数は以下のようにして知ることができます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(len(class_obj[0]))
print(len(class_obj[1]))
print(len(class_obj[2]))

3
3
3


3個ある`CLASS_OBJ`の要素には，それぞれ3個の要素があることがわかりました．
それではこの要素にはどのようなキーが含まれているでしょうか．
ディクショナリ型オブジェクトのキーを取得するには`keys()`メソッドを使います．
使い方は
```
オブジェクト名.keys()
```
になります．
オブジェクト`class_obj[0]`から`class_obj[2]`のキーを取得するには以下のようにします．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(class_obj[0].keys())
print(class_obj[1].keys())
print(class_obj[2].keys())

dict_keys(['@id', '@name', 'CLASS'])
dict_keys(['@id', '@name', 'CLASS'])
dict_keys(['@id', '@name', 'CLASS'])


3個ある`CLASS_OBJ`の要素（オブジェクト`class_obj[0]`から`class_obj[2]`）には，
それぞれ`@name`，`@id`，`CLASS`の3個のキーがあることがわかりました．
すなわち，以下のような構造になっています．

```
"CLASS_OBJ": [
    {
        "@id": "cat01", 
        "@name": "年齢別（５歳階級、４区分）、男女別人口", 
        "CLASS": [
            {
・・・途中省略・・・
    {
        "@id": "cat02", 
        "@name": "秘匿地域・合算地域有り", 
        "CLASS": [
            {
・・・途中省略・・・
    {
        "@id": "area", 
        "@name": "東京都", 
        "CLASS": [
            {
・・・以下省略・・・
```

これらのキーの値を見てみましょう．
まずは`@name`キーです．
`@name`キーの値を取得するには以下のようにします．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(class_obj[0]['@name'])
print(class_obj[1]['@name'])
print(class_obj[2]['@name'])

年齢別（５歳階級、４区分）、男女別人口
秘匿地域・合算地域有り
東京都


`@name`は属性の名称ないしは説明事項（表章事項）を表しているようです．
すなわち，
`class_obj[0]`は統計表の表章事項のうち年齢別・男女別の階級を，
`class_obj[1]`は統計表の表章事項のうち秘匿地域・合算地域を，
`class_obj[2]`は統計表の表章事項のうち地域情報を表していそうです．

これを表にまとめてみましょう．

class_obj|`['@name']`
-|-
`class_obj[0]`|年齢別（５歳階級、４区分）、男女別人口
`class_obj[1]`|秘匿地域・合算地域有り
`class_obj[2]`|東京都

次に`@id`キーの値も見てみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(class_obj[0]['@id'])
print(class_obj[1]['@id'])
print(class_obj[2]['@id'])

cat01
cat02
area


`@id`は表章事項を表しているキーのような役割のようです．
これも表に追加します．

class_obj|`['@name']`|`['@id']`
-|-
`class_obj[0]`|年齢別（５歳階級、４区分）、男女別人口|`cat01`
`class_obj[1]`|秘匿地域・合算地域有り|`cat02`
`class_obj[2]`|東京都|`area`

# CLASSの確認

次に`CLASS`キーです．
`CLASS`キーは後ろにスクエアブラケット「`[]`」が続いていることから，
値としてリスト型オブジェクトが入っています．
例えば年齢別（５歳階級、４区分）、男女別人口（`class_obj[0]`）の場合，
以下のような形になっています．

```
{
    "@id": "cat01", 
    "@name": "年齢別（５歳階級、４区分）、男女別人口", 
    "CLASS": [
        {
            要素
        },
        {
            要素
        },
        {
            要素
        },
・・・以下省略・・・
    ]
}
```

`CLASS`キーの要素はブレース「`{}`」で囲まれていることから，
これらの要素はディクショナリ型オブジェクトであることがわかります．

# 年齢別・男女別カテゴリについて

年齢別・男女別のカテゴリを表している
`class_obj[0]["CLASS"]`の要素を見てみます．
このキー`CLASS`の値にはいくつの要素があるのでしょうか．
`len()`関数を用いて確認してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
len(class_obj[0]['CLASS'])

60

60個ありました．
すべてを表示するには数が多いため，
最初の要素のみ表示してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(json.dumps(class_obj[0]['CLASS'][0], ensure_ascii = False, indent = 4))

{
    "@code": "T000573001",
    "@name": "総数、年齢「不詳」含む",
    "@level": "2",
    "@unit": "人"
}


2番目と最後の要素も確認してみましょう

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(json.dumps(class_obj[0]['CLASS'][1], ensure_ascii = False, indent = 4))
print(json.dumps(class_obj[0]['CLASS'][59], ensure_ascii = False, indent = 4))

{
    "@code": "T000573002",
    "@name": "総数０～４歳",
    "@level": "2",
    "@unit": "人"
}
{
    "@code": "T000573060",
    "@name": "女７５歳以上",
    "@level": "2",
    "@unit": "人"
}


`class_obj[0]["CLASS"]`は年齢別・男女別のカテゴリを表しているようです．
すなわち，「年齢別（５歳階級，４区分），男女別人口」というカテゴリの
すべてのカテゴリがリストとして格納されており，その数は60個であることがわかります．
e-Statからダウンロードした年齢別（５歳階級，４区分），男女別人口の定義書（**T000573.pdf**）に合わせてみると，
キー`@level`は項目の「階層」を表しており，このデータ（年齢別（５歳階級，４区分），男女別人口）の場合はすべて`2`になっています．
キー`@code`は年齢別・男女別のコードを表しており，定義書の「連番」に相当します．
キー`@name`は定義書の「項目名」であり，
キー`@unit`は「単位」に相当します．

これも表に追加してみましょう．

class_obj|`['@name']`|`['@id']`|`['CLASS']`
-|-
`class_obj[0]`|年齢別（５歳階級、４区分）、男女別人口|`cat01`|年齢別・男女別のカテゴリ（60個），`@code`は「連番」，`@name`は「項目名」，`@unit`は「単位」
`class_obj[1]`|秘匿地域・合算地域有り|`cat02`|
`class_obj[2]`|東京都|`area`|

# 秘匿・合算地域情報について

次に`class_obj[1]["CLASS"]`を確認してみます．
`class_obj[1]`は秘匿地域・合算地域の情報でした．
これは以下のようになっています．

```
{
    "@id": "cat02", 
    "@name": "秘匿地域・合算地域有り", 
    "CLASS": [
```

`len()`関数を用いて`class_obj[1]["CLASS"]`の要素数を確認します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
len(class_obj[1]['CLASS'])

3

3個あることがわかりました．
次にそれぞれの要素を見ていきます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(json.dumps(class_obj[1]['CLASS'][0], ensure_ascii = False, indent = 4))
print(json.dumps(class_obj[1]['CLASS'][1], ensure_ascii = False, indent = 4))
print(json.dumps(class_obj[1]['CLASS'][2], ensure_ascii = False, indent = 4))

{
    "@code": "1",
    "@name": "無し",
    "@level": "1"
}
{
    "@code": "2",
    "@name": "合算",
    "@level": "1"
}
{
    "@code": "3",
    "@name": "秘匿",
    "@level": "1"
}


この結果から，`class_obj[1]["CLASS"][0]`から`class_obj[1]["CLASS"][2]`には
それぞれ`@level`，`@code`，`@name`の3個の要素があることがわかります．
`@level`はすべて1であることから，無視しても構わないでしょう．
`@code`は数値が入っており，`@name`は説明が入っているようです．
すなわち，`@code`が`1`の場合には秘匿地域ではない通常の地域で，
`2`の場合は合算地域（他の秘匿地域の値が合算されている）であることを示し，
`3`の場合は秘匿地域であることを示しています．
これも表に追加します．

class_obj|`['@name']`|`['@id']`|`['CLASS']`
-|-
`class_obj[0]`|年齢別（５歳階級、４区分）、男女別人口|`cat01`|年齢別・男女別のカテゴリ（60個），`@code`は「連番」，`@name`は「項目名」，`@unit`は「単位」
`class_obj[1]`|秘匿地域・合算地域有り|`cat02`|`@code`が`1`：通常の地域，`2`：合算されている地域，`3`：秘匿地域
`class_obj[2]`|東京都|`area`|

>秘匿地域とは，人口や世帯が極端に少ない地域の場合，統計情報から個人の情報が推定できることを避けるため意図的に値を隠している（0が入っている）地域のことを示します．統計値の矛盾を避けるために秘匿地域の情報は別の地域に合算されており，秘匿地域の値が合算されている地域のことを合算地域と呼びます．基本的に秘匿地域と合算地域は隣接しています．

後で使うため，秘匿・合算地域情報をディクショナリ型のオブジェクト`htk_obj`に代入しておきます．

In [0]:
htk_obj = class_obj[1]['CLASS']

#  地域情報について

最後に`class_obj[2]`を確認します．
`class_obj[2]`は地域情報を表しています．
これは以下のようになっています．

```
{
    "@id": "area", 
    "@name": "東京都", 
    "CLASS": [
        {
            "@code": "13101", 
            "@level": "1", 
            "@name": "千代田区"
        }, 
        {
            "@code": "131010010", 
            "@level": "2", 
            "@name": "千代田区丸の内"
        }, 
```

地域情報も年齢別・男女別カテゴリや秘匿・合算地域と同様に`@id`，`@name`，`CLASS`の3個の要素から構成されています．
こちらも`len()`関数を用いて`class_obj[2]["CLASS"]`の要素数を確認します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
len(class_obj[2]['CLASS'])

6781

要素数は6781個もありました．
東京都の小地域データには6781個の地域があるようです．
チェックのため最初の3個だけを表示してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(json.dumps(class_obj[2]['CLASS'][0], ensure_ascii = False, indent = 4))
print(json.dumps(class_obj[2]['CLASS'][1], ensure_ascii = False, indent = 4))
print(json.dumps(class_obj[2]['CLASS'][2], ensure_ascii = False, indent = 4))

{
    "@code": "13101",
    "@name": "千代田区",
    "@level": "1"
}
{
    "@code": "131010010",
    "@name": "千代田区丸の内",
    "@level": "2"
}
{
    "@code": "13101001001",
    "@name": "千代田区丸の内１丁目",
    "@level": "3"
}


この結果から，`class_obj[2]["CLASS"]`の各要素には
それぞれ`@level`，`@code`，`@name`の3個の要素があることがわかります．
`@level`は1から3の値が，
`@code`は数値が，
`@name`は地名が入っているようです．
これらの値はデータは何を示しているのでしょうか．

冒頭で見せたe-Statからダウンロードした東京都の小地域データ（**tblT000573C13101.xlsx**）と対比してみると，
キー`@level`は列`HYOSYO`と同じであり，小地域のレベルを示していると考えられます．
すなわちキー`@level`が`1`だと区（市区町村）レベルを，`2`だと町レベルを，`3`だと丁目レベルをそれぞれ表しています．
これに対応して，キー`@code`は全国の町丁目を表すユニーク値である列`KEY_CODE`，すなわち行政コードであり，
キー`@name`が実際の市区町村・町丁目の名称となっています．この結果を表に追加しましょう．

class_obj|`['@name']`|`['@id']`|`['CLASS']`
-|-
`class_obj[0]`|年齢別（５歳階級、４区分）、男女別人口|`cat01`|年齢別・男女別のカテゴリ（60個），`@code`は「連番」，`@name`は「項目名」，`@unit`は「単位」
`class_obj[1]`|秘匿地域・合算地域有り|`cat02`|`@code`が`1`：通常の地域，`2`：合算されている地域，`3`：秘匿地域
`class_obj[2]`|東京都|`area`|`@level`は小地域のレベル（`HYOSYO`と同一），`@code`は行政コード（`KEY_CODE`と同一），`@name`は市区町村・町丁目の名称

後で使うため，地域情報をディクショナリ型のオブジェクト`area_obj`に代入しておきます．

In [0]:
area_obj = class_obj[2]['CLASS']

これでメタデータの解釈が終わりました．

# 表の列名データの作成

実際にデータをダウンロードした後，どのような型式に整形するのがよいでしょうか．
一つの考え方としては，
通常のe-Statを利用したときにダウンロードできる型式に近いものにするのがよいのかもしれません．

e-Statからダウンロードした小地域データは以下のような構造をしています．
**tblT000573C13101.xlsx**を参照してみてください．

KEY_CODE|HYOSYO|CITYNAME|NAME|HTKSYORI|HTKSAKI|GASSAN|T000573001|T000573002|省略|T000573060
-------:|-----:|:-------|:---|:-------|------:|-----:|---------:|---------:|:--:|---------:
|||||||総数、年齢「不詳」含む|総数０〜４歳|・・・|女７５歳以上
13101|1|千代田区|||||47115|1688|・・・|2882
131010010|2|千代田区|丸の内||||X|X|・・・|X
13101001001|3|千代田区|丸の内１丁目|秘匿地域|140||X|X|・・・|X

左の列から`KEY_CODE`（行政コード），`HYOSYO`（小地域のレベル），`CITYNAME`（市区町村名）と`NAME`（小地域名），`HTKSYORI`（秘匿地域・合算地域の有無），`HTKSAKI`（秘匿地域の場合どの地域に合算しているか），`GASSAN`（合算地域の場合どの地域を合算しているか），`T000573001`から`T000573060`（年齢別・男女別のカテゴリ）となっています．

この表の1行目，すなわち列名の部分をリスト型のオブジェクト`col_name`に入れてみます．
まずは`KEY_CODE`から`GASSAN`までです．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
col_name = [
    u'KEY_CODE', u'HYOSYO', u'CITYNAME', u'NAME', u'HTKSYORI', u'HTKSAKI', u'GASSAN'
]

単純な文字列のリストです．
クオーテーションマークの前の「`u`」は，
この文字列がユニコードであることを明示しています．

次に男女別・年齢別のカテゴリ名の部分です．
`T000573001`から`T000573060`までは`class_obj[0]['CLASS'][0]['@code']`から
`class_obj[0]['CLASS'][59]['@code']`に入っているため，
この部分を`col_name`の後ろに追加します．

0から59までの要素番号は1ずつ大きくなるため，
`for`ループを使ってすべての要素を順にアクセスします．
`for`ループの書式は以下です．

```
for i in range(60):
    print(class_obj[0]['CLASS'][i]['@code'])
```

`for`ループはリスト型などのシーケンスオブジェクトから順次要素を取りだし，
要素値を変数に入れて要素の数だけループを行います．
ここではシーケンスオブジェクトとして`range`関数を用いています．

`range`関数は引数で与えられた範囲の整数値をリストとして返す関数です．
引数がひとつだけの場合は0から値-1までの整数を返します．
この場合は0から59（60ではありません）になります．
`for`文は`range`関数で作成された値を一つずつ変数`i`に入れループを実行します．
つまり，ループの中で`class_obj[0]['CLASS'][i]['@code']`とすることにより，
`class_obj[0]['CLASS'][0]['@code']`から`class_obj[0]['CLASS'][59]['@code']`
まで順次アクセスすることができます．

リストに値を追加するには，リスト型オブジェクトに対して`append`メソッドを使います．
以下で実際にコードを実行してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
for i in range(60):
    col_name.append(class_obj[0]['CLASS'][i]['@code'])

値が入っているか確認してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#col_name

年齢別・男女別カテゴリの数は60個ですが，
これは`class_obj[0]['CLASS']`の要素数になります．
すなわち`len(class_obj[0]['CLASS'])`で要素数を取得できます．
これを利用してコードを変更します．

In [0]:
col_name = [
    u'KEY_CODE', u'HYOSYO', u'CITYNAME', u'NAME', u'HTKSYORI', u'HTKSAKI', u'GASSAN'
]
for i in range(len(class_obj[0]['CLASS'])):
    col_name.append(class_obj[0]['CLASS'][i]['@code'])

こちらも値が入っているか確認してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#col_name

```
[u'KEY_CODE',
 u'HYOSYO',
 u'CITYNAME',
 u'NAME',
 u'HTKSYORI',
 u'HTKSAKI',
 u'GASSAN',
 u'T000573001',
 u'T000573002',
・・・以下省略・・・
```

うまくいったようです．

# 表の枠組み（行と列）の作成

次に統計表本体の枠組みを作ります．
統計表の1行は一つの小地域を表していますので，
小地域コードをキー名，統計表の列を値としたディクショナリ型で表すことができます．
具体的には以下のようなディクショナリ型のオブジェクト`stat_tab`を作成してみます．

```
stat_tab = {
    小地域コード（"KEY_CODE"）: {
        小地域コード（"KEY_CODE"）: 値,
        自治体レベル（"HYOSYO"）: 値,
        市区町村名（"CITYNAME"): 値,
        小地域名（"NAME"): 値,
        秘匿・合算の有無（"HTKSYORI"）: 値,
        秘匿先（"HTKSAKI"）: 値,
        合算元（"GASSAN"）: 値,
        年齢別・男女別人口のコード（class_obj[0]["CLASS"][i]["@code"]の値）: 値,
            これを年齢別・男女別人口のカテゴリ数だけ繰り返す（60個）
    },
・・・これを小地域の数繰り返す・・・
}
```       

試しに最初の小地域のディクショナリオブジェクトを作成します．
最初の小地域のメタ情報は`class_obj[2]["CLASS"][0]`に入っており，
小地域コードは`class_obj[2]["CLASS"][0]["@code"]`になります．
以下のコードを実行することにより，
小地域コードをキーに持つディクショナリの要素を作成することができます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab = {}
stat_tab[class_obj[2]['CLASS'][0]['@code']]={}

まず空のディクショナリ型オブジェクト`stat_obj`を作成します．
次に`stat_tab`の要素である`class_obj[2]["CLASS"][0]["@code"]`の中に，
さらに空のディクショナリオブジェクトを作成します．
ここで要素のキーとして参照している`class_obj[2]["CLASS"][0]["@code"]`は
小地域のコード番号になります．

最初の小地域のコード番号は`13101`になりますので，
`stat_tab[class_obj[2]["CLASS"][0]["@code"]]`は
`stat_tab["13101"]`を示しています．

オブジェクト`stat_tab`は空のディクショナリ型オブジェクトとして作成したばかりですので，
キー`stat_tab["13101"]`は存在していません．
存在していないキーに対してアクセスを行うと，
そのキーを持つ要素が生成されます．
うまく要素が生成されたか確認してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab

{'13101': {}}

オブジェクト`stat_tab`は要素が一つだけのディクショナリ型のオブジェクトになっています．
要素のキーは最初の小地域のコードである`13101`で，
その値は空白のディクショナリ型オブジェクトです．

次に空白のディクショナリ型オブジェクトに入れるための要素`f_obj`を作成しましょう．
実はこの要素のキーは，統計表の列名を表しているリスト型オブジェクト`col_name`とまったく同じですので，
これを利用して`f_obj`を作成します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
f_obj = {i: u'' for i in col_name}

オブジェクト名`f_obj`の後ろにブレースが来ていることから，
`f_obj`はディクショナリ型オブジェクトとして作られています．
このオブジェクトの値は内包表記という仕組みを用いて作成しています．
ディクショナリ型オブジェクトの要素は
キーと値をコロンでつなげ，カンマで区切って羅列します．
この場合，キーにはオブジェクト`i`の値が入り，
値としては`u""`が入ります．
`u""`ははユニコード型の文字列として表現したヌル文字（空白の文字）を表現しています．
実際にデータを取ってこないと値はわからないため，
初期値としてヌル文字を入れいています．

オブジェクト`i`は，後ろに続く`for i in col_name`によって
オブジェクト`col_name`の要素を順に`i`に入れていきます．

オブジェクト`f_obj`がちゃんと作成されたか確認してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#f_obj

```
{u'CITYNAME': u'',
 u'GASSAN': u'',
 u'HTKSAKI': u'',
 u'HTKSYORI': u'',
 u'HYOSYO': u'',
 u'KEY_CODE': u'',
 u'NAME': u'',
 u'T000573001': u'',
 ・・・途中省略・・・
 u'T000573060': u''}
```

順番は一部変わっていますが，
`KEY_CODE`から`GASSAN`までと`T000573001`から`T000573060`までが
オブジェクト`f_obj`の要素として加わっており，
値はすべてヌル文字になっていることが確認できます．

それではオブジェクト`stat_tab`に`f_obj`を加えていきましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab[class_obj[2]['CLASS'][0]['@code']] = f_obj

`stat_tab`を確認します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#stat_tab

```
{u'13101': {u'CITYNAME': u'',
  u'GASSAN': u'',
  u'HTKSAKI': u'',
  u'HTKSYORI': u'',
  u'HYOSYO': u'',
  u'KEY_CODE': u'',
  u'NAME': u'',
  u'T000573001': u'',
  u'T000573002': u'',
・・・以下省略・・・
```

上記のコードは以下のようにも書けます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab[class_obj[2]['CLASS'][0]['@code']] = {i: u'' for i in col_name}

この操作を小地域の数だけ繰り返します．
小地域の数は`class_obj[2]["CLASS"]`の要素の数だけありますので，
`for`文を使ってループ処理します．

In [0]:
for u in range(len(class_obj[2]['CLASS'])):
    stat_tab[class_obj[2]['CLASS'][u]['@code']] = {i: u'' for i in col_name}

`stat_tab`の行数と，最初の要素の列数を確認してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
print(len(stat_tab))
print(len(stat_tab['13101']))

6781
67


行数が6781行，列数が67列のオブジェクトができました．
次はこのオブジェクトに統計データ取得用APIで取得した値を埋め込んでいきます．

# データ本体の取得

それではデータ本体を取ってきましょう．
データを取得するには統計データ取得用APIを使います．

APIマニュアル**「3.4 統計データ取得」**より，
統計データ取得用APIではアプリケーションIDの`appId`と
統計表IDの`statsDataId`が必須パラメータです．
これらのパラメータをディクショナリ`param`に格納してAPIへ渡します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
param = {
    'appId': 'c8fbd72d8f58aedf4ca3106034601d0ea83e9a69',
    'statsDataId': tab_info['@id'],
}
api_param = urllib.parse.urlencode(param)
data_obj = urllib.request.urlopen(APIdata_URL + api_param)
response = data_obj.read()
json_data = json.loads(response)

以下のコードを実行すると，
統計データ取得用APIの結果を`APIstats.json`として保存することができます．
保存した結果を[JSON EDITOR ONLINE (http://jsoneditoronline.org/)](http://jsoneditoronline.org/)
などで表示してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
f = codecs.open('APIstats.json', 'w', 'utf-8')
f.write(json.dumps(json_data, ensure_ascii = False, indent = 4))
f.close()

```
{
    "GET_STATS_DATA": {
        "STATISTICAL_DATA": {
・・・途中省略・・・
        },
        "PARAMETER": {
            "LANG": "J", 
            "STATS_DATA_ID": "T00057313", 
            "START_POSITION": 1, 
            "DATA_FORMAT": "J", 
            "METAGET_FLG": "Y"
        }, 
        "RESULT": {
            "STATUS": 0, 
            "DATE": "2016-10-23T07:26:04.563+09:00", 
            "ERROR_MSG": "正常に終了しました。"
        }
    }
}        
```

結果をチェックします．

ルートタグは`GET_STATS_DATA`で，
その下に子タグとして`STATISTICAL_DATA`と
`PATAMETER`と`RESULT`があります．
APIマニュアルの**「4.4.2 STATISTICAL_DATAタグ」**より，
`RESULT`はAPIリクエストの結果，
`PARAMETER`はリクエスト時に指定したパラメータを出力したものです．
入手したいデータ本体は`STATISTICAL_DATA`に入っています．

`STATISTICAL_DATA`を詳細に見ていきましょう．

```
"STATISTICAL_DATA": {
    "DATA_INF": {
    },
・・・途中省略・・・
    "RESULT_INF": {
・・・途中省略・・・
    }, 
    "TABLE_INF": {
・・・途中省略・・・
    },
    "CLASS_INF": {
・・・途中省略・・・
    }
}
```

`STATISTICAL_DATA`には子タグとして`DATA_INF`，`RESULT_INF`，`TABLE_INF`，`CLASS_INF`の4個のタグがあります．
APIマニュアルより，
`TABLE_INF`は統計表の情報が，
`CLASS_INF`は統計データのメタ情報が，
`RESULT_INF`は取得したデータの件数などがそれぞれ格納されており，
データ本体は`DATA_INF`に格納されています．
`DATA_INF`を詳細に見ていきましょう．

```
"DATA_INF": {
    "NOTE": {
・・・途中省略・・・
    },
    "VALUE": [
・・・途中省略・・・
    ]
},
```

`DATA_INF`には`NOTE`と`VALUE`の2個のタグがあります．
e-Stat APIマニュアル「**4.4.2. STATISTICAL_DATAタグ（31ページ）**」によると，
`NOTE`は特殊文字の凡例とあります．
`NOTE`は以下の定義になっています．

```
"NOTE": {
    "@char": "-",
    "$": "当該数値がないもの"
},
```


つまり，統計の値が入るべきところに「`-`」が入っている場合，
該当する値がないということを表しています．

`VALUE`タグを見てみましょう．
APIマニュアルより，このタグは統計情報（セル）の情報が入っているとあります．
`VALUE`タグはスクエアブラケット「`[]`」が続いているため，
要素としてリスト型オブジェクトが入っています．
具体的には以下のようになっています．

```
"VALUE": [
    {
        "@cat02": "1", 
        "@cat01": "T000573001", 
        "$": "47115", 
        "@area": "13101", 
        "@unit": "人"
    }, 
・・・途中省略・・・
    {
        "@cat02": "1", 
        "@cat01": "T000573015", 
        "$": "159", 
        "@area": "13209038002", 
        "@unit": "人"
    }
]
```

`VALUE`の要素はブラケットで囲まれているため，
ディクショナリ型オブジェクトであることがわかります．
このディクショナリ型オブジェクトには5個の要素があり，
`@cat02`，`@cat01`，`$`，`@area`，`@unit`となっています．

APIマニュアルの**「4.2.2 STATISTICAL_DATAタグ（31ページ）」**より，
`@cat01`と`@cat02`は分類事項コード，
`@area`は地域事項コード，
`@unit`は単位となっています．
また，`$`は値そのもの（ここでは小地域の男女別・年齢別の人口）を表しています．

`@cat01`，`@cat02`，`@area`はメタデータを取得したときにまとめた以下の表に対応しています．
つまり，`@cat01`の値は男女別・年齢別カテゴリのコード番号そのものを表し，
`@cat02`が１の場合は合算・秘匿地域がない通常の地域，2の場合は合算されている地域，
3の場合は秘匿地域を表しています．
`@area`は小地域の行政コードです．

class_obj|`['@name']`|`['@id']`|`['CLASS']`
-|-
`class_obj[0]`|年齢別（５歳階級、４区分）、男女別人口|`cat01`|年齢別・男女別のカテゴリ（60個），`@code`は「連番」，`@name`は「項目名」，`@unit`は「単位」
`class_obj[1]`|秘匿地域・合算地域有り|`cat02`|`@code`が`1`：通常の地域，`2`：合算されている地域，`3`：秘匿地域
`class_obj[2]`|東京都|`area`|`@level`は小地域のレベル（`HYOSYO`と同一），`@code`は行政コード（`KEY_CODE`と同一），`@name`は市区町村・町丁目の名称

この情報を元にしてオブジェクト`stat_tab`に小地域のデータを入れていきましょう．

統計データ取得用APIで取得したデータのうち，
小地域ごとの男女別・年齢別人口が格納されているのは
`json_data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']`以下になります．
必要な部分だけを切り出し，リスト型オブジェクト`stat_obj`に格納します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_obj = json_data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

`stat_obj`の要素数をチェックして，
切り出しができているか確認します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
len(stat_obj)

100000

小地域のデータは10万件しか取得できていないようです．

# データの一部分しか取れていない?

東京都の小地域データの場合
データの件数は406860件あるはずですが，
およそ4分の1しかデータが取れていません．
この原因はどこにあるのでしょうか．

統計データ取得用APIの結果は
`RESULT_INF`タグに書かれていますので，
これを詳細に見てみます．

```
"RESULT_INF": {
    "FROM_NUMBER": 1,
    "TO_NUMBER": 100000,
    "TOTAL_NUMBER": 406860,
    "NEXT_KEY": 100001
}, 
```

APIマニュアルの**「3.4. 統計データ取得」**の`limit`の説明（8ページ）では，
データ取得件数を指定するパラメータ`limit`の指定がない場合は
最大10万件を取得するとあります．
今回対象としている東京都の小地域データの総数は406860件であり，
キー`TOTAL_NUMBER`にも書かれています．
また，これはデータ本体を取得しなくても，
統計表情報取得用APIの`OVERALL_TOTAL_NUMBER`からあらかじめ知ることができます．

データの開始位置を示す`FROM_NUMBER`には`1`が，
終了位置を示す`TO_NUMBER`には`100000`が入っています．
これはすべてのデータ件数が406860件あるうち，
今回のデータ取得では1件目から10万件目までしか取得できていないことを示しています．
APIマニュアルには`limit`の指定がない場合は最大10万件を取得するとありますが，
試しに10万件より大きな値を`limit`にした場合でも
10万件までしか取得できませんでした．

これでは東京都の全データを取得できませんので，
何回かに分けてすべてのデータを取るようにしなければなりません．

データ取得用APIに渡すパラメータを再検討しましょう．
パラメータは以下のようになっていました．

```
param = {
    'appId': 'c8fbd72d8f58aedf4ca3106034601d0ea83e9a69',
    'statsDataId': tab_info['@id'],
}
```

これにデータの取得開始位置を表す`startPosition`を加えます．
APIマニュアルの**「3.4. 統計データ取得（8ページ）」**より，
`startPosition`は前回取得したデータの`NEXT_KEY`タグの値を指定するとあります．
`NEXT_KEY`タグは`RESULT_INF`の中にありますので，
変更後のパラメータは以下になります．
変更したパラメータの確認までしてみましょう

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
param = {
    'appId': 'c8fbd72d8f58aedf4ca3106034601d0ea83e9a69',
    'statsDataId': tab_info['@id'],
    'startPosition': json_data['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['NEXT_KEY']
}

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
param

{'appId': 'c8fbd72d8f58aedf4ca3106034601d0ea83e9a69',
 'statsDataId': 'T00057313',
 'startPosition': 100001}

`startPosition`の追加ができており，
取得開始位置も100001番目からになっています．
このパラメータをURLエンコードして100001件目から20000件目までの統計表データを取得します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
api_param = urllib.parse.urlencode(param)
data_obj = urllib.request.urlopen(APIdata_URL + api_param)
response = data_obj.read()
json_data = json.loads(response)

100001件目から200000件目まで取得できているか，
`RESULT_INF`を確認してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
json_data['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']

{'TOTAL_NUMBER': 406860,
 'FROM_NUMBER': 100001,
 'TO_NUMBER': 200000,
 'NEXT_KEY': 200001}

`FROM_NUMBER`と`TO_NUMBER`により，
100001件目から200000件目まで取得できていることがわかります．
次回のデータ取得は`NEXT_KEY`より200001件目からの10万件になります．

ここで，いま取得した100001件目から200000件目までのデータを，
1件目から100000件目までのデータを表しているオブジェクト`stat_obj`に追加します．
オブジェクト`stat_obj`はリスト型のオブジェクトであるため，
リスト型オブジェクトに要素を追加する`append`メソッドを使います．
`append`メソッドの使い方は以下になります．
```
リスト型オブジェクト.append(追加する要素)
```
リスト型オブジェクト`stat_obj`に2回目に取得したデータの`VALUE`キー以下の要素を追加しますので，
このようなコードになります．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#for i in json_data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']:
for i in tqdm(json_data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']):
    stat_obj.append(i)

100%|██████████| 100000/100000 [00:00<00:00, 2048430.09it/s]


`for`は`json_data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']`の要素数だけループします．
`['VALUE']`には追加の10万件のデータが入っているためループの回数は10万回になります．
このとき，変数`i`には`['VALUE']`の要素がひとつずつ入ります．
これを`append`メソッドを使って`stat_obj`に追加しています．

追加後のデータ件数も確認しましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
len(stat_obj)

200000

20万件になっています．
データの追加読み込みと結合はうまくいっているようです．

ここまでのコードをまとめてみましょう．

In [0]:
# startPositionの初期値を設定する
start_position = 1

# リスト型オブジェクトstat_objを作成する
stat_obj = []

# データを10万件ずつ読み出すループ
while True:
    
    # APIに渡すパラメータの設定
    param = {
        'appId': 'c8fbd72d8f58aedf4ca3106034601d0ea83e9a69',
        'statsDataId': tab_info['@id'],
        'startPosition': start_position
    }

    # APIの呼び出し，ディクショナリ型オブジェクトへの変換
    api_param = urllib.parse.urlencode(param)
    data_obj = urllib.request.urlopen(APIdata_URL + api_param)
    response = data_obj.read()
    json_data = json.loads(response)
     
    # VALUEキー以下をstat_objに追加する
    for i in json_data['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']:
        stat_obj.append(i)
    
    # NEXT_KEYキーがある場合，次回のループ用のstart_positionを設定する
    # NEXT_KEYがない場合はwhileループを抜ける
    if 'NEXT_KEY' in json_data['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']:
        start_position = json_data['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['NEXT_KEY']
    else:
        break

まず`startPosition`の初期値として`1`を入れ，
データの格納先であるリスト型オブジェクト`stat_obj`を生成します．
次にデータをAPIから読み込み，
読んだデータをリスト型オブジェクト`stat_obj`へ格納するループに入ります．

`while True:`ループはループを無限に繰り返すループです．
APIのパラメータ設定としてキー`startPosition`に`start_position`の値を代入し，
URLエンコードしてAPIにアクセスします．
取得したデータはキー`VALUE`以下をリスト型オブジェクト`stat_obj`に追加します．

取得したデータにキー`NEXT_KEY`がある場合にはまだ読み込まれていないデータがあるため，
`start_position`にキー`NEXT_KEY`の値を代入して次のループを行います．
すべてのデータを読み込んでしまうと`NEXT_KEY`自体が無くなるため，
`break`を使って`while`ループを抜けます．

オブジェクト`stat_obj`にデータの全件（406860件）が入ったか確認しましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
len(stat_obj)

406860

これで無事に東京都の小地域データのすべてが取得できました．

# 人口データの埋め込み

それでは`stat_tab`に`stat_obj`の内容を埋め込んでいきましょう．
まずは手始めに`stat_obj`の最初の要素を`stat_tab`に埋め込みます．
`stat_obj`の最初の要素は以下のようになっています．

```
{
    "@cat02": "1", 
    "@cat01": "T000573001", 
    "$": "47115", 
    "@area": "13101", 
    "@unit": "人"
}, 
```

一方，`stat_tab`の各要素はこのような形になっています．

```
{
    "13101": {
        "CITYNAME": "",
        "GASSAN": "",
        "HTKSAKI": "",
        "HTKSYORI": "",
        "HYOSYO": "",
        "KEY_CODE": "",
        "NAME": "",
        "T000573001": "",
        "T000573002": "",
・・・途中省略・・・
}
```

`stat_obj`の最初の要素は，`stat_tab`のどの要素に入れればよいでしょうか．
`stat_tab`のキーは小地域のコードになっていますので，
`stat_obj`の中で小地域コードを示す要素値を取得することにより，
`stat_tab`のどの要素にアクセスすれば良いのかわかります．
`stat_obj`で小地域コードを表すキーは`@area`になりますので，
```
stat_tab[stat_obj[0]['@area']]
```
とすることで，`
stat_obj`の最初の要素`[0]`の地域コードが示している
`stat_tab`の要素にアクセスできます．

さらに`stat_tab`の要素`KEY_CODE`には`stat_obj`の`@area`キーの値を入れたいので
以下のようにします．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab[stat_obj[0]['@area']]['KEY_CODE'] = stat_obj[0]['@area']

同様に，`stat_tab`の要素`HTKSYORI`には`stat_obj`の`@cat02`キーの値を入れます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab[stat_obj[0]['@area']]['HTKSYORI'] = stat_obj[0]['@cat02']

次に，`stat_obj`の`@cat01`キーの値の人口（`$`の値）を，
`stat_tab`の同じキーの値に入れます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab[stat_obj[0]['@area']][stat_obj[0]['@cat01']] = stat_obj[0]['$']

うまく代入されているか確認してみましょう．
`stat_obj`の最初の要素は地域コード`13101`ですので，
`stat_tab`の要素`13101`を表示してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#stat_tab['13101']

```
{u'CITYNAME': u'',
 u'GASSAN': u'',
 u'HTKSAKI': u'',
 u'HTKSYORI': u'1',
 u'HYOSYO': u'',
 u'KEY_CODE': u'13101',
 u'NAME': u'',
 u'T000573001': u'47115',
 u'T000573002': u'',
 u'T000573003': u'',
・・・以下省略・・・
}
```

`HTKSYORI`，`KEY_CODE`，`T000573001`に正しい値が入っています．
この操作をループを使って行えば，
`stat_obj`のすべての要素を`stat_tab`に入れることができます．

In [0]:
for i in tqdm(range(len(stat_obj))):
    stat_tab[stat_obj[i]['@area']]['KEY_CODE'] = stat_obj[i]['@area']
    stat_tab[stat_obj[i]['@area']]['HTKSYORI'] = stat_obj[i]['@cat02']
    stat_tab[stat_obj[i]['@area']][stat_obj[i]['@cat01']] = stat_obj[i]['$']

100%|██████████| 406860/406860 [00:00<00:00, 705087.27it/s]


うまくできているか確認しましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#stat_tab['13202002004']

```
{u'CITYNAME': u'',
 u'GASSAN': u'',
 u'HTKSAKI': u'',
 u'HTKSYORI': u'',
 u'HYOSYO': u'',
 u'KEY_CODE': u'13202002004',
 u'NAME': u'',
 u'T000573001': u'2069',
 u'T000573002': u'95',
 u'T000573003': u'89',
 ・・・以下省略・・・
}
```

うまくいっているようです．

小地域名などの地域情報がまだ入っていませんので，
次はこの情報を入れていきます．

# 地域情報の埋め込み

地域情報はディクショナリ型オブジェクト`area_obj`に格納してあります．
人口データと同様に`area_obj`の要素の数だけ`stat_tab`にアクセスして
値を入れていきます．

オブジェクト`area_obj`の一番最初の要素は`area_obj[0]`になり，
`area_obj[0]['@code']`が一番最初の要素の小地域コードになります．
従って，以下のようにすれば`area_obj`の最初の要素の市区町村・町丁目の名称を
`stat_tab`の同一小地域コードの`NAME`に入れることができます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab[area_obj[0]['@code']]['NAME'] = area_obj[0]['@name']

本来の統計表では`NAME`が表しているのは町丁目名のみですが，
e-Stat APIから取得してきたデータには市区町村名と町丁目名が合わさっているため，
ここでは`NAME`の値として市区町村名と町丁目名が合わさっているものを入れます．
その代わり，`CITYNAME`に`tab_info['TABLE_SUB_CATEGORY1']`に保持している都道府県名を入れます．
さらに`HYOSYO`に小地域のレベルである`area_obj[0]['@level']`を入れます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
stat_tab[area_obj[0]['@code']]['CITYNAME'] = tab_info['TABLE_SUB_CATEGORY1']
stat_tab[area_obj[0]['@code']]['HYOSYO'] = area_obj[0]['@level']

地域情報がうまく格納できたか確認してみましょう

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#print(json.dumps(stat_tab['13101'], ensure_ascii = False, indent = 4, sort_keys = True))

```
{
    "CITYNAME": "東京都", 
    "GASSAN": "", 
    "HTKSAKI": "", 
    "HTKSYORI": "1", 
    "HYOSYO": "1", 
    "KEY_CODE": "13101", 
    "NAME": "千代田区", 
    "T000573001": "47115", 
    "T000573002": "1688", 
    "T000573003": "1639", 
    "T000573004": "1728", 
・・・以下省略・・・
}
```

`CITYNAME`には都道府県名が，
`HYOSYO`には地域のレベルが，
`NAME`には市区町村名と（おそらく）小地域名が入っていることが確認できました．
`for`ループを使ってすべての小地域に値を入れましょう．

In [0]:
for i in tqdm(range(len(area_obj))):
    stat_tab[area_obj[i]['@code']]['CITYNAME'] = tab_info['TABLE_SUB_CATEGORY1']
    stat_tab[area_obj[i]['@code']]['NAME'] = area_obj[i]['@name']
    stat_tab[area_obj[i]['@code']]['HYOSYO'] = area_obj[i]['@level']

100%|██████████| 6781/6781 [00:00<00:00, 672854.87it/s]


これですべての地域情報が`stat_tab`に格納できました．

# データをcsvファイルとして書き出す

データがすべて揃いましたので，これをcsvファイルに書き出します．
csvに書き出すには`csv`モジュールの`csv.writer`オブジェクトと`writerows`メソッドが便利です．
`csv.writer`オブジェクトと`writeraws`メソッドは，
シーケンス型のデータを指定されたデリミタ（カンマなど）で区切り
ファイルに書き出すものです．

シーケンス型のオブジェクトにはリスト型オブジェクトなどがありますが，
`stat_tab`などで使っているディクショナリ型オブジェクトはシーケンス型のオブジェクトではありません．
このため，`csv.write`を使って`stat_tab`を直接csvファイルに書き出すことができません．
ディクショナリ型オブジェクトである`stat_tab`をリスト型のオブジェクトに変換する必要があります．

# `stat_tab`をリスト型オブジェクトに変換する

まずは`stat_tab`の最初の要素値をリスト型オブジェクトに変換しましょう．
リストの1行分を保持するリスト型オブジェクト`one_row`を作成します．
次に`one_row`の要素として，`stat_tab`の要素値の一つを統計表の順番で加えていきます．

ディクショナリ型オブジェクトの要素には順番の概念がないため，
明示的に順番を指定しないと行によって順番が異なってしまう可能性があります．
このため，統計表の列の順番で`one_row`に埋め込んでいきます．
統計表の列はリスト型オブジェクト`col_name`に保存してありますので，
`col_name`の要素の順番で`stat_tab`の要素にアクセスして`row`に埋め込んでいけば
順番が一定になります．

`stat_tab`の最初の要素値は`stat_tab['13101']`になりますので，
以下のコードで1行目のリストを作ることができます．
ファイル出力を考えて，`encode`メソッドを使って要素値をUTF-8に変換します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
one_row = []
for i in col_name:
    one_row.append(stat_tab['13101'][i].encode('utf-8'))

`one_row`の内容を確認しましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#print(json.dumps(one_row, ensure_ascii = False, indent = 4))

```
[
    "13101", 
    "1", 
    "東京都", 
    "千代田区", 
    "1", 
    "", 
    "", 
    "47115", 
    "1688", 
    "1639", 
    "1728", 
    "1724", 
    "2771", 
・・・以下省略・・・
]
```

**tblT000573C13101.xlsx**の最初のデータと見比べてみてください．
列の値と順番が同じになっているのが確認できます．

上記のコードはforループを使っていますが，
内包表記で表すと以下のようになります．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
one_row = []
one_row = [stat_tab['13101'][i].encode('utf-8') for i in col_name]

こちらも`one_raw`の内容を確認してみます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#print(json.dumps(one_row, ensure_ascii = False, indent = 4))

```
[
    "13101", 
    "1", 
    "東京都", 
    "千代田区", 
    "1", 
    "", 
    "", 
    "47115", 
    "1688", 
    "1639", 
    "1728", 
    "1724", 
    "2771", 
・・・以下省略・・・
]
```

`for`ループを使用した場合と同一の結果になりました．

>Pythonでは一般的に`for`ループを使うよりも内包表記を使う方が処理速度が2倍程度速くなります．
その代わり内包表記は慣れないとわかりにくくなってしまいます．
スピードと慣れのトレードオフになりますが，
あまりに凝りすぎてプログラムがわかりにくくなってしまうのも問題です．
ほどほどに使っておきましょう．

`stat_tab`の最初の要素をリスト型オブジェクト`one_row`の要素にすることができました．
`for`ループを使って`stat_tab`のすべての要素をリスト型にします．

`one_row`は1行分のデータで，これを別のリスト型オブジェクト`tab_all`に`append`メソッドを使って追加していきます．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
tab_all = []
one_row = []
for u in stat_tab.keys():
    one_row = [stat_tab[u][i].encode('utf-8') for i in col_name]
    tab_all.append(one_row)

`tab_all`の1行目を確認してみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#print(json.dumps(tab_all[0], ensure_ascii = False, indent = 4))

```
[
    "132220120", 
    "2", 
    "東京都", 
    "東久留米市小山", 
    "1", 
    "", 
    "", 
    "5570", 
    "205", 
    "257", 
    "272", 
・・・以下省略・・・
]
```

千代田区ではない東久留米のデータが1行目に来ています．
ディクショナリ型オブジェクトは順番が不定になりますので
このような結果になっています．
チェックはできませんが，多分大丈夫でしょう．

`for`ループを使ってディクショナリ型オブジェクトの`stat_tab`を
リスト型オブジェクトの`tab_all`に変換しましたが，
これも内包表記を使って表すことができます．

以下が内包表記の例です．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
tab_all = []
tab_all = [[stat_tab[u][i].encode('utf-8') for i in col_name] for u in stat_tab.keys()]

まず`tab_all`をリスト型オブジェクトとして生成します．

`tab_all = [[`の部分は，リスト型オブジェクト`tab_all`の要素として
更にリストを入れることを表しています．
つまり`tab_all`はリストのリストになります．

`tab_all`の要素は`[stat_tab[u][i] for i in col_name]`になります．
これは値として`stat_tab`の要素を一つずつ入れることを表しています．
`stat_tab`のどの要素にアクセスするかは添え字として使われている`u`と`i`の値で決まります．

`u`は`for u in stat_tab.keys()`となっており，
ディクショナリ型オブジェクト`stat_tab`のキーの一覧（リスト型になる）
を一つずつ返します．
また，`i`は`for i in col_name`となっており，
リスト型オブジェクト`col_name`の要素を一つずつ返します．

`[stat_tab[u][i] for i in col_name]`の部分は
リスト`col_name`の要素を一つずつ`i`に入れ，`stat_tab`の一行分をリストにします．
一行分の処理が終わったら`u`が`col_name`の2番目の要素になり，
再度`[stat_tab[u][i] for i in col_name]`が繰り返されます．

このようにしてディクショナリ型オブジェクト`stat_tab`を
リスト型オブジェクト`tab_all`に変換することができます．

`tab_all`の適当な要素をチェックしてみましょう．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
#print(json.dumps(tab_all[600], ensure_ascii = False, indent = 4))

```
[
    "13222001001", 
    "3", 
    "東京都", 
    "東久留米市上の原１丁目", 
    "1", 
    "", 
    "", 
    "1270", 
    "13", 
    "10", 
    "19", 
・・・以下省略・・・
]
```

こちらもうまくいっているようです．

# csv.writerを使ってcsvとして書き出す

リスト型オブジェクトになった`tab_all`と
列名を表すリスト型オブジェクト`col_name`を`csv.writer`を使って
csvファイルに書き出しましょう

In [0]:
# 確認用コードにつき，最終プログラムからは削除します
f = codecs.open(tab_info['@id']+'.txt', 'w')
csv_writer = csv.writer(f)
csv_writer.writerow(col_name)
csv_writer.writerows(tab_all)
f.close()

まず`codecs.open`を使ってデータ保存用のファイルを書き込みモードで開き，
ファイルポインタを`f`として取得します．
データ保存用のファイル名は`tab_info['@id']+'.txt'`となっています．
`tab_info['@id']`には統計表情報取得用APIから取得した統計表IDが入っています．
この値に`.txt`をつけたものをデータ保存用のファイル名にしています．

ファイルポインタ`f`に対して`csv.writer`を使って
`csv.writer`オブジェクトの`csv_writer`を作成します．
この`csv_writer`オブジェクトに`writerow`メソッドを使って
列名を表すリスト型オブジェクト`col_name`をカンマ分けで出力し，
さらに表の本体を表すリスト型オブジェクト`tab_all`を`writerows`メソッドで出力します．
最後にファイルポインタ`f`を`close`メソッドを使って閉じます．

保存された`T00057313.txt`をエクセルで開いて中身を確認してみましょう．

# 欠損値の処理

エクセルで中身を見てみると，いくつかのデータが欠損「`-`」になっていることがわかります．
このため，このcsvファイルをArcGISにインポートすると
小地域ごと，男女別・年齢別人口が文字列として扱われてしまいます．
これではコロプレスマップなどができませんので，
適切な形で欠損値を処理する必要があります．

処理の方法はいくつか考えられますが，
最も単純なのは欠損値を0にする方法です．
ここではこの方法で処理をしてみましょう．

欠損値を0で置き換えるための一番単純な方法は，
リスト型オブジェクト`tab_all`の要素に一つずつアクセスし，
値が欠損値を示す「`-`」の場合に0で更新するという方法です．

`tab_all`は二重のリストになっているため，
二重の`for`ループでアクセスすればよいでしょう．
値が欠損値かどうかは`if`を用いて判断します．
これは以下のようなコードになります．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します

# 二重のforループでifを使った例
for i in tqdm(range(len(tab_all))):
    for u in range(len(tab_all[0])):
        if tab_all[i][u] == '-':
            tab_all[i][u] = 0

100%|██████████| 6781/6781 [00:00<00:00, 99556.76it/s]


`i`のループは`tab_all`の要素数だけ繰り返します．
すなわち小地域の数だけループすることになります．
`u`のループは`tab_all[0]`の要素数だけループします．
これはデータの列の数に相当しますので，67回のループになります．
Python以外の言語に慣れている人にはわかりやすいコードになっています．

以下のコードで欠損値「`-`」を0に置き換えたものを
`統計表ID_loop.txt`という名前で保存します．

エクセルで内容を確認すると，
欠損値をあらわす「`-`」がすべて0になっていることがわかります．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します

# 統計表ID_loop.txtとして書き出し
f = codecs.open(tab_info['@id']+'_loop.txt', 'w')
csv_writer = csv.writer(f)
csv_writer.writerow(col_name)
csv_writer.writerows(tab_all)
f.close()

`if`を使わないで欠損値を0に置き換えてみましょう．
置き換えには`replace`メソッドを使います．
`replace`メソッドは第一引数の文字を第二引数で置き換えますが，
文字に対してのみ使用できます．
リスト型オブジェクト`tab_all`の各要素は文字列と数値が混ざっています．
このため，文字列の要素（つまり欠損値である「`-`」が入っている要素）は`replace`メソッドが使えますが，
数値の要素の場合にはエラーが出てしまいます．
これを避けるために，`tab_all`のすべての要素を文字列に変換し，
`replae`メソッドが使えるようにします．
数値を文字列に変換するには`str`関数を使います．

以下のコードは`replace`メソッドを使った例です．
まず`tab_all`を作り直します．
次に二重の`for`ループの中で`tab_all`の各要素を`str`関数を用いて文字列に変換し，
`replace`メソッドを使って欠損値を0に置き換えます．
そのあとで，結果を`統計表ID_replace.txt`という名前で保存します．

In [0]:
# 確認用コードにつき，最終プログラムからは削除します

# 欠損値の入っているtab_allを作り直す
tab_all = []
tab_all = [[stat_tab[u][i].encode('utf-8') for i in col_name] for u in stat_tab.keys()]

#二重のforループでreplaceメソッドを使った例
for i in range(len(tab_all)):
    for u in range(len(tab_all[0])):
        tab_all[i][u] = str(tab_all[i][u]).replace('-', '0')

# 統計表ID_replae.txtとして書き出し
f = codecs.open(tab_info['@id']+'_replace.txt', 'w')
csv_writer = csv.writer(f)
csv_writer.writerow(col_name)
csv_writer.writerows(tab_all)
f.close()

最後は内包表記で作成したものです．

In [0]:
# 欠損値の入っているtab_allを作り直す
tab_all = []
tab_all = [[stat_tab[u][i].encode('utf-8') for i in col_name] for u in stat_tab.keys()]

#内包表記バージョン
tab_all = [[str(tab_all[u][i]).replace('-', '0') for i in range(len(tab_all[0]))] for u in range(len(tab_all))]

f = codecs.open(tab_info['@id']+'.txt', 'w')
csv_writer = csv.writer(f)
csv_writer.writerow(col_name)
csv_writer.writerows(tab_all)
f.close()

新しい`tab_all`の値は，文字列に変換した`tab_all`の中の欠損値「`-`」を0に置き換えたものになります．
`tab_all`の各要素へのアクセスは二重の内包表記で表しており，
内側のループは`tab_all[0]`の要素の数，外側のループは`tab_all`の要素の数となっています．

以上でe-Stat APIから小地域のデータを取得し，
csv型式で保存することができました．
ここまでのプログラムを`estat_api_test.py`として用意しておきました．

ここまでのプログラムはArcPyの機能を使っていないため，
ArcGISが動いていない環境でも実行することができます．

# ArcPyによるコロプレス作成

# ArcPyの設定

ここからはArcPyを用いた作業になります．
ArcPyを使うにはArcPyモジュールをインポートする必要があります．
ArcPyモジュールを使用するには，有効なArcGISライセンスが必要になります．

- `arcpy`：ArcGISの機能にアクセスするためのモジュール

ArcPy本体のインポートと，
ArcGISの環境設定に関するクラスをenvという別名でアクセスできるようにするための設定を行います．


import sys, os
import arcpy
from arcpy import env

次に作業用のワークスペースを設定します．
ワークスペースとは，ArcGISがファイルを読み書きする標準の場所のことを表します．
そのためには，オブジェクト`env.workspace`にワークスペースにしたいディレクトリの絶対パスを
クオーテーションで囲んで指定します．

絶対パスではディレクトリの区切りに`\`マークを使いますが，
Pythonでの`\`マークはエスケープシーケンスという特別な役割を持つ文字になりますので，
`\`を含む文字列はraw文字列として指定し，エスケープを解除した方がよいでしょう．
文字列をraw文字列として指定するには，クオーテーションの前に"`r`"をつけます．

具体的には以下のようにします．
この例では，Tドライブの`work`ディレクトリをワークスペースにします．

# クオーテーションの中は実際のワークスペースを指定してください
env.workspace = r'T:\work'

# ファイルジオデータベースの作成

先ほど作成した小地域データをインポートするためのファイルジオデータベースを作成します．
作成するジオデータベースの名称は`estat.gdb`とし，変数`out_gdb`にファイル名を入れておきます．
また，小地域データのファイル名も変数`in_data`に入れておきましょう．

out_gdb = 'estat.gdb'
in_tab = 'T00057313.txt'
in_base, in_ext = os.path.splitext(in_tab)
out_tab = out_gdb + '\\' + in_base

次に実際にファイルジオデータベースを作成します．
ファイルジオデータベースを作成するためには，
ArcPyの`CreateFileGDB_management`関数を使います．
使い方は以下になります．
```
arcpy.CreateFileGDB_management(作成する場所, 作成するファイルジオデータベースの名前)
```
作成する場所はワークスペース，作成するファイルジオデータベースの名前は変数`out_gdb`になるため，
以下のコードになります．

arcpy.CreateFileGDB_management(env.workspace, out_gdb)

これでファイルジオデータベースが作成されました．

# csvファイルをファイルジオデータベースへインポート

次に作成したファイルジオデータベースにcsvファイルをインポートします．
ファイルジオデータベースにcsvファイルをインポートするには
ArcPyの`TableToTable_conversion`関数を使います．
`TableToTable_conversion`関数の使い方は
```
arcpy.TableToTable_conversion(読み込むデータ, 格納先のファイルジオデータベース名, ジオデータベース内のテーブル名）
```
になります．

読み込むデータは`in_data`，
格納先のファイルジオデータベース名は`out_gdb`，
書き込むテーブル名は`in_base`になるため，
以下のコードになります．

arcpy.TableToTable_conversion(in_tab, out_gdb, in_base)

ファイルジオデータベース`estat.gdb`にテーブル`T00057313`ができているかArcMapで確認しましょう．

# シェープファイルのマージ

東京23区の小地域シェープファイルを用意しました．
これをマージします．
シェープファイルのマージは`Merge_management`関数を使います．
`Merge_management`関数の使い方は以下になります．

```
arcpy.Merge_management(マージするシェープファイルをリストで指定, マージ後のシェープファイル名)
```

マージするシェープファイルはリスト型オブジェクトとしてまとめて指定します．
ArcPyにはワークスペース内にあるシェープファイルをリスト型オブジェクトとして返す
`ListFeatureClasses`関数があります．
`ListFeatureClasses`関数の使い方は以下です．

```
arcpy.ListFeatureClasses(ワイルドカード)
```

ワイルドカードを使うことにより，ワークスペース内にあるシェープファイルのうち，
任意の条件のファイル名を持つシェープファイルのみをリスト化することができます．
例えば，ワークスペース内にあるシェープファイルのうち，
`h22ka`で始まるシェープファイルのみをリスト化したい場合はこのようにします．

```
arcpy.ListFeatureClasses('h22ka*')
```

コードをまとめると以下のようになります．

shapes = arcpy.ListFeatureClasses('h22ka*')
arcpy.Merge_management(shapes, out_gdb + r'\tokyo23')

まず現在のワークスペース内にある`h22ka`で始まるシェープファイルをリスト化し，
リスト型オブジェクト`shapes`に格納します．
次に`shapes`の要素に対して`Merge_management`関数を用いてマージを行います．
マージの結果は`out_gdb`の中に`tokyo23`という名前のフィーチャとして保存します．

ArcPyを用いてファイルジオデータベース内のフィーチャやテーブルにアクセスするときには，
ジオデータベース名とフィーチャ名の区切りに`\`記号を使います．
Pythonでは`\`記号は特別な意味を持ちますので，
raw文字列（クオーテーションの前に`r`をつける）として指定するのがよいでしょう．

マージの結果をArcMapで確認してみましょう．

# シェープファイルへのテーブル結合

次はフィーチャへのテーブル結合です．
テーブル結合はフィーチャとテーブルの共通する値を持つ属性をキーにして，
フィーチャにテーブルを結合する操作です．
この場合，フィーチャとテーブルのキーとなる属性は同じデータタイプである必要があります．

マージしたフィーチャ名を変数`feature_name`に格納しておきましょう．

feature_name = out_gdb + r'\tokyo23'

テーブル結合の対象となるフィーチャは，
変数`feature_name`で表されているジオデータベース内の`tokyo23`，
結合するテーブルは同じくジオデータベース内の`T00057313`になります．
テーブル名は`out_tab`に格納されています．

どちらのデータにも小地域を表す`KEY_CODE`がありますが，
フィーチャ側の`KEY_CODE`はTEXT型（文字列型）で，
テーブル側の`KEY_CODE`はDouble型（数値の倍精度実数型）になっています．
データタイプが異なるため，このままではテーブル結合ができません．
両方のデータタイプをそろえる必要があります．

ここでは，フィーチャ側の`KEY_CODE`をDouble型に変換し，
これをキーとしてテーブル側の`KEY_CODE`と結合します．
すでに存在している属性のデータタイプを変更することはできないため，
テーブル側の`KEY_CODE`と同一のデータタイプを持つ新しい属性をフィーチャ側に作成し，
その属性にシェープファイルの属性`KEY_CODE`の値を入れます．

新しい属性を作るには`AddField_management`関数を使います．
`AddField_management`関数の使い方は，

```
arcpy.AddField_management(属性を追加するテーブル名, 追加する属性名, 追加する属性のデータタイプ,)
```
になります．

フィールドはフィーチャに追加しますので，
属性を追加するテーブル名は追加するフィーチャの名前そのもの，
つまり変数`feature_name`になります．
追加したい属性名は`KC`，データタイプは`DOUBLE`とします．

以下が実際のコードになります．

arcpy.AddField_management(feature_name, 'KC', 'DOUBLE')

結果をArcMapで確認しましょう．

次にシェープファイルの属性`KEY_CODE`の値を新しく作成した`KC`にコピーします．
`KEY_CODE`のデータタイプはテキスト，`KC`のデータタイプは数値ですが，
自動的にコピー先のデータタイプに変換されて値が格納されます．

データのコピーにはフィールド演算の機能を使います．
ArcPyでフィールド演算を行うのは`CalculateField_management`関数になります．
`CalculateField_management`関数の書式は
```
arcpy.CaclulateField_management(計算対象のテーブル名, 計算結果を格納する属性名, 計算式, 計算式の書式)
```
になります．

実際のコードは以下になります．

arcpy.CalculateField_management(feature_name, 'KC', '!KEY_CODE!', 'PYTHON')

属性`KC`に属性`KEY_CODE`の値を代入します．
属性の値を参照する際には属性名を`!`で囲む必要があります．
この書式はPython型式なので，計算式の書式を`PYTHON`とします．

結果を確認してみましょう．

次はテーブル結合です．

テーブル結合を行うためには`JoinField_management`関数を使います．
通常のテーブル結合では，テーブル結合されたフィーチャないしシェープファイルの属性テーブルは
画面上ではテーブルが結合されているように見えるのですが，
実際には結合されたデータが作成されたわけではありませんでした．
このため，テーブル結合の後にデータのエクスポートをする必要がありました．
これに対し，`JoinField_management`関数はフィーチャにテーブルを完全に結合します．

`JoinField_management`書式は以下になります．

```
JoinField_management(結合されるフィーチャ名, フィーチャ側のキー名, 結合するテーブル名, テーブル側のキー名)
```

変数`feature_name`で表されているジオデータベース内の`tokyo23`と，
変数`out_tab`で表されているジオデータベース内の`T00057313`を
それぞれ`KC`と`KEY_CODE`で結合します．
コードは以下になります．

arcpy.JoinField_management(feature_name, 'KC', out_tab, 'KEY_CODE')

データ量が多いためテーブル結合に少々時間がかかります．
結合が終わったら結果を確認してみてください．

以上，ArcPyを用いた部分を`estat_arcpy.py`として用意しておきました．

# おわりに

駆け足でしたが，ArcPyからe-Stat APIを使うための解説をしてみました．
ほとんどの部分がe-Stat APIをPythonaでどのように使うかという説明でしたが，
うまくe-Stat APIを使うことで仕事や研究が効率的に進んでもらえればと思います．

ArcPyはPythonのモジュールというのが表向きの姿です．
ArcGISをインストールするとPythonの実行環境もインストールされますが，
インストールされたPythonはバージョンが少し古いなど，
少々使いにくい部分もあります．

本格的にPythonプログラミングをしたいのであれば，
anacondaなどのオールインワンパッケージをインストールするのが良いと思います．
anacondaのPython環境は最新バージョンであり，
pandas，scipy，numpy，scikit-learnなどの統計計算，数値計算，機械学習モジュール
などもすぐに使えるようになっています．
anacondaのPythonでArcPyを使えるように設定すれば，
かなり強力な処理系になります．

実際にPythonやArcPyのプログラムを開発する環境としては，
Jupyter Notebookやeclipse+PyDevがおすすめです．

Jupyter Notebookは探索的なプログラミングをするのに向いていますし，
eclipseはIDEなのでフレームワークを使ったプログラミングに適しています．

今回のこのテキストはJupyter Notebookを使って作成しています．
プログラムとメモを同時に入力できるため，
処理のプロセスを明確にすることができますし，
このようなテキストを作成するには最適な環境だと思います．

おすすめの方法としては，Jupyter Notebookを使って特定の機能を持つプログラムを探索的に作成し，
フレームワークのパーツを作っていきます．
フレームワークのパーツができたらeclipseを使ってそれを組み合わせ，
大規模なプログラムを作成するという方法です．

プログラミング言語もプログラミング環境もさまざまなものがあり，
それぞれ長所と短所があります．
うまく組み合わせてeffective programmingを実践したいと思います．